<a href="https://colab.research.google.com/github/MdShahadatHossainbd/Model/blob/main/OptimizeCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create CNN Model and Optimize it using Keras Tuner

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 3.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=074334eccc5bd116f615bc4b1245ae5766a5f53d7ca195fb023aeeaec048eb18
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=2814ed7b886cf0d9826e69ce13a9926bc68bfb082d2975dad778df8afd090484
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.3.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [11]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [12]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [14]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [15]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.4457 - accuracy: 0.8413 - val_loss: 0.3215 - val_accuracy: 0.8797
Epoch 2/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.2925 - accuracy: 0.8928 - val_loss: 0.3045 - val_accuracy: 0.8893
Epoch 3/3
1688/1688 [==============================] - 13s 7ms/step - loss: 0.2405 - accuracy: 0.9107 - val_loss: 0.2721 - val_accuracy: 0.9023


Epoch 1/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.4901 - accuracy: 0.8261 - val_loss: 0.4058 - val_accuracy: 0.8550
Epoch 2/3
1688/1688 [==============================] - 14s 9ms/step - loss: 0.3676 - accuracy: 0.8666 - val_loss: 0.3852 - val_accuracy: 0.8548
Epoch 3/3
1688/1688 [==============================] - 14s 9ms/step - loss: 0.3372 - accuracy: 0.8756 - val_loss: 0.3770 - val_accuracy: 0.8655


Epoch 1/3
1688/1688 [==============================] - 19s 11ms/step - loss: 0.5551 - accuracy: 0.8184 - val_loss: 0.4221 - val_accuracy: 0.8447
Epoch 2/3
1688/1688 [==============================] - 18s 11ms/step - loss: 0.4011 - accuracy: 0.8510 - val_loss: 0.3744 - val_accuracy: 0.8595
Epoch 3/3
1688/1688 [==============================] - 18s 11ms/step - loss: 0.3751 - accuracy: 0.8602 - val_loss: 0.4165 - val_accuracy: 0.8540


Epoch 1/3
1688/1688 [==============================] - 14s 8ms/step - loss: 0.6471 - accuracy: 0.7783 - val_loss: 0.4807 - val_accuracy: 0.8258
Epoch 2/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.4382 - accuracy: 0.8376 - val_loss: 0.4340 - val_accuracy: 0.8433
Epoch 3/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.4128 - accuracy: 0.8475 - val_loss: 0.4291 - val_accuracy: 0.8460


Epoch 1/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.5420 - accuracy: 0.8078 - val_loss: 0.4347 - val_accuracy: 0.8417
Epoch 2/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.4033 - accuracy: 0.8538 - val_loss: 0.4025 - val_accuracy: 0.8572
Epoch 3/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.3694 - accuracy: 0.8646 - val_loss: 0.4157 - val_accuracy: 0.8550


INFO:tensorflow:Oracle triggered exit


In [16]:
model=tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 48)        1248      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        13856     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                495648    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 511,082
Trainable params: 511,082
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.2061 - accuracy: 0.9240 - val_loss: 0.2623 - val_accuracy: 0.9023
Epoch 5/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1722 - accuracy: 0.9357 - val_loss: 0.2822 - val_accuracy: 0.9000
Epoch 6/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1443 - accuracy: 0.9476 - val_loss: 0.2835 - val_accuracy: 0.9095
Epoch 7/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1223 - accuracy: 0.9538 - val_loss: 0.2947 - val_accuracy: 0.9115
Epoch 8/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1022 - accuracy: 0.9631 - val_loss: 0.3203 - val_accuracy: 0.9115
Epoch 9/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0846 - accuracy: 0.9691 - val_loss: 0.3391 - val_accuracy: 0.9068
Epoch 10/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0742 - accuracy: 0.9723 - val_loss: 0.3925 - val_a